In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://github.com/allenai/S2AFF/raw/main/data/gold_affiliation_annotations.csv')

In [3]:
data = []
for i, row in df.iterrows():
    elt = {}
    elt['label'] = row.original_affiliation
    rors = []
    for r in eval(row.labels):
        if 'ror.org' in r:
            rors.append(r.replace('https://ror.org/', ''))
    elt['ror'] = rors
    if rors:
        data.append(elt)

In [4]:
import requests, json

def match(conditions):
    return requests.post("http://localhost:5004/match", json=conditions).json()


def compute_precision_recall(data):

    index_prefix = 'matcher'
    local = True
    match_type = 'ror'

    nb_TP, nb_FP, nb_FN = 0, 0, 0
    false_positive, false_negative = [], []
    for ix, d in enumerate(data):
        if ix%100==0:
            print(ix, end=',')
        if d.get(match_type):
            res = match({'query': d['label'], 'year':'2020', 'type': match_type, 'index_prefix': index_prefix})
            for x in res['results']:
                if x in d[match_type]:
                    nb_TP += 1
                else:
                    nb_FP += 1
                    false_positive.append(d)
            for x in d[match_type]:
                if x not in res['results']:
                    nb_FN += 1
                    false_negative.append(d)

    precision = nb_TP / (nb_TP + nb_FP)
    recall = nb_TP / (nb_TP + nb_FN)
    res = {'precision' : precision, 'recall' : recall}
    return {'res': res, 'false_positive': false_positive, 'false_negative': false_negative} 


In [5]:
res = compute_precision_recall(data)

0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,

In [6]:
res['res']

{'precision': 0.9248737373737373, 'recall': 0.624467178175618}

In [9]:
res['false_positive'][0]

{'label': 'The Laboratory of Photochemistry, Center for Molecular Science, Institute of Chemistry, Chinese Academy of Sciences, Beijing 100080, PR China',
 'ror': ['034t30j35']}

In [ ]:
# The affiliation-matcher also gives 048y1rc66 which seems ok...